In [1]:
import pickle
import fastf1 as ff1
import pandas as pd
import numpy as np

from fastf1.core import Session
from typing import List, Iterable

import f1_pitstop_advisor.idea_one.data_processing_utils as utils

from sklearn.pipeline import make_pipeline

In [2]:
with open("ig_sessions.pickle", "rb") as file:
    sessions = pickle.load(file)

In [37]:
# Organize sessions by circuit
data_by_circuit = {}
for session in sessions:
    circuit_key =  session.session_info["Meeting"]["Circuit"]["Key"]
    if circuit_key not in data_by_circuit.keys():
        data_by_circuit[circuit_key] = []
    data_by_circuit[circuit_key].append(session)

In [38]:
# Get lap data combined with weather data
studied_sessions = data_by_circuit[63] 
data_list = []
for session in studied_sessions:
    session_data = utils.get_lap_data_with_weather(session)
    utils.add_z_score_for_laps(session_data, inplace=True)
    session_data = session_data.convert_dtypes()
    data_list.append(session_data)

data = pd.concat(data_list, ignore_index=True)



In [39]:
data["IsPitLap"] = np.isnat(data["PitInTime"])

In [40]:
data.columns

Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
       'Compound', 'TyreLife', 'FreshTyre', 'Team', 'LapStartTime',
       'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
       'FastF1Generated', 'IsAccurate', 'AirTemp', 'Humidity', 'Pressure',
       'Rainfall', 'TrackTemp', 'WindDirection', 'WindSpeed', 'LapTimeSeconds',
       'LapTimeZScore', 'IsPitLap'],
      dtype='object')

In [41]:
selected_columns = [
    "LapTimeZScore", "IsPitLap", "Compound", "TyreLife", "FreshTyre", # Lap info
    "AirTemp", "Humidity", "Pressure", "Rainfall", "TrackTemp", "WindDirection", "WindSpeed" # Weather data
]
filtered_data = data.loc[:, selected_columns]

In [44]:
final_data = pd.get_dummies(filtered_data)

In [45]:
final_data

,LapTimeZScore,IsPitLap,TyreLife,FreshTyre,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,Compound_HARD,Compound_MEDIUM,Compound_SOFT
0,-0.271308,True,4,False,23.9,26,1010.4,False,29.0,13,0.3,False,False,True
1,-0.525218,True,5,False,23.8,26,1010.4,False,29.0,357,0.5,False,False,True
2,-0.473811,True,6,False,23.8,29,1010.2,False,28.8,13,0.4,False,False,True
3,-0.451286,True,7,False,23.8,31,1010.4,False,28.7,50,0.3,False,False,True
4,-0.417769,True,8,False,23.8,33,1010.4,False,28.5,316,0.4,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4409,-0.423008,True,26,True,26.9,44,1008.4,False,30.4,8,1.5,False,True,False
4410,-0.393133,True,27,True,26.9,44,1008.4,False,30.5,33,1.9,False,True,False
4411,-0.402422,True,28,True,26.9,43,1008.4,False,30.3,44,1.2,False,True,False
4412,-0.330371,True,29,True,26.9,43,1008.4,False,30.4,44,1.4,False,True,False
